In [2]:
%%time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

CPU times: user 1.26 s, sys: 234 ms, total: 1.49 s
Wall time: 3.17 s


In [4]:
%%time
with open('/content/drive/MyDrive/Colab Notebooks/train.txt','r') as train:
  train=train.readlines()
  train_values=[]
  train_labels=[]

  for i in train:
    columns=[0 for i in range(100000)]
    train_labels.append(int(i[0]))
    i=i.replace("0\t","")
    i=i.replace("1\t","")
    i=i.replace("\n","")
    new_list=list(map(int,i.split(" ")[:-1]))
    for i in new_list:
      columns[i-1]=1
    train_values.append(columns)
  x=np.array(train_values)
  y=np.array(train_labels)

CPU times: user 9.25 s, sys: 1.2 s, total: 10.4 s
Wall time: 10.5 s


In [5]:
%%time
with open('/content/drive/MyDrive/Colab Notebooks/test.txt','r') as test:
  test=test.readlines()
  test_values=[]

  for i in test:
    columns=[0 for i in range(100000)]
    new_list=list(map(int,i.split(" ")[:-1]))
    for i in new_list:
      columns[i-1]=1
    test_values.append(columns)
  test=np.array(test_values)

CPU times: user 4.33 s, sys: 432 ms, total: 4.76 s
Wall time: 4.77 s


In [6]:
%%time
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1,random_state=42)

CPU times: user 97.2 ms, sys: 165 ms, total: 263 ms
Wall time: 263 ms


**Naive Bayes**

First model that i have used is Naive bayes classifier

In [48]:
%%time
#for choosing number of components in principal component analysis i have take a  for loop aand choosen best numbe rof components
for i in range(100,720,100):
  pca=PCA(n_components=i)
  x_train_pca=pca.fit_transform(x_train)
  x_test_pca=pca.transform(x_test)
  test_pca=pca.transform(test)
  pca_NB = BernoulliNB()
  pca_NB.fit(x_train_pca,y_train)
  y_pca_pred=pca_NB.predict(x_test_pca)
  print("F1 score for number of components : ", str(i))
  print("F1->",f1_score(y_pca_pred,y_test))


F1 score for number of components :  100
F1-> 0.8750000000000001
F1 score for number of components :  200
F1-> 0.8
F1 score for number of components :  300
F1-> 0.8
F1 score for number of components :  400
F1-> 0.6666666666666667
F1 score for number of components :  500
F1-> 0.6666666666666667
F1 score for number of components :  600
F1-> 0.6666666666666667
F1 score for number of components :  700
F1-> 0.6666666666666667
CPU times: user 7min 2s, sys: 35.1 s, total: 7min 38s
Wall time: 4min 16s


Resampling the data using random over sampler and then performing PCA and appling naive bayes classifier

In [7]:
%%time
sampling = RandomOverSampler(random_state=0)
x_resampled, y_resampled = sampling.fit_resample(x, y)
print(sorted(Counter(y_resampled).items()))

[(0, 722), (1, 722)]
CPU times: user 698 ms, sys: 4.77 s, total: 5.47 s
Wall time: 5.33 s


In [9]:
%%time
x_sampling_train, x_sampling_test, y_sampling_train, y_sampling_test = train_test_split(x_resampled, y_resampled, test_size=0.1,random_state=42)

CPU times: user 266 ms, sys: 1.35 s, total: 1.62 s
Wall time: 2.37 s


In [12]:
%%time
#applied pca
pca_sampling=PCA(n_components=300)
x_train_sampling_pca=pca_sampling.fit_transform(x_sampling_train)
x_test_sampling_pca=pca_sampling.transform(x_sampling_test)

CPU times: user 1min 6s, sys: 3.11 s, total: 1min 9s
Wall time: 38.3 s


In [14]:
%%time
# trained the model
pca_sampling_NB = BernoulliNB()
pca_sampling_NB.fit(x_train_sampling_pca,y_sampling_train)
y_pca_sampling_pred=pca_sampling_NB.predict(x_test_sampling_pca)
print("F1 score of bernoulli classifier after resampling the data:")
f1_score(y_pca_sampling_pred,y_sampling_test)

F1 score of bernoulli classifier after resampling the data:
CPU times: user 18.5 ms, sys: 6.04 ms, total: 24.6 ms
Wall time: 21.8 ms


0.9241379310344827

In [34]:
#Using random oversampler is trying to overfit the model. Though the model develops an f1 score of 0.84 on train set, when the test set is given to predict the values the model couldn't even predict 50% values of the values correctly. 

Using PCA without using random oversampler and then applying naive bayes classifier

In [26]:
%%time
#trained the model without applying random over sampling
pca=PCA(n_components=300)
x_train_pca=pca.fit_transform(x_train)
x_test_pca=pca.transform(x_test)
test_pca=pca.transform(test)

CPU times: user 46.1 s, sys: 4.56 s, total: 50.7 s
Wall time: 28.7 s


In [27]:
%%time
pca_NB = BernoulliNB()
pca_NB.fit(x_train_pca,y_train)
y_pca_pred=pca_NB.predict(x_test_pca)
print("F1 score of bernoulli classifier:")
f1_score(y_pca_pred,y_test)

F1 score of bernoulli classifier:
CPU times: user 16.8 ms, sys: 4.88 ms, total: 21.7 ms
Wall time: 16.5 ms


0.75

In [17]:
%%time
print("Classification report:")
print(classification_report(y_pca_pred, y_test))

Classification report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        75
           1       0.56      1.00      0.71         5

    accuracy                           0.95        80
   macro avg       0.78      0.97      0.84        80
weighted avg       0.97      0.95      0.96        80

CPU times: user 11.5 ms, sys: 0 ns, total: 11.5 ms
Wall time: 21.5 ms


In [25]:
%%time
test_out = list(pca_NB.predict(test_pca))
file=open('/content/drive/MyDrive/Colab Notebooks/output_NB_pca.txt','w')
for i in test_out:
  file.write(str(i)+'\n')
file.close()
#f1 score for test set came out to be 75 and can be found in miner

**Decision Tree Classifier**

Second model that i have used is decision tree classifier

In [18]:
%%time
df_x=pd.DataFrame(data=x)
df_x.var().describe()

CPU times: user 983 ms, sys: 8.53 ms, total: 991 ms
Wall time: 1.07 s


count    100000.000000
mean          0.008893
std           0.011081
min           0.000000
25%           0.001250
50%           0.004981
75%           0.012359
max           0.190213
dtype: float64

In [19]:
%%time
#removed the columns with variance less than threshold value of 0.012359
columns=df_x.columns
new_columns=[]
for i in range(len(columns)):
  if df_x[i].var()>=0.012359:
    new_columns.append(columns[i])

CPU times: user 22.4 s, sys: 490 ms, total: 22.8 s
Wall time: 23 s


In [20]:
%%time
#performed train test split and trained the model
df_x=df_x[new_columns]
df_x_train, df_x_test, y_DT_train, y_DT_test = train_test_split(df_x, y, test_size=0.1,random_state=42)
DT=DecisionTreeClassifier()
DT.fit(df_x_train,y_DT_train)
y_DT_pred=list(DT.predict(df_x_test))

CPU times: user 4.52 s, sys: 16.6 ms, total: 4.54 s
Wall time: 4.55 s


In [21]:
%%time
#calculated f1 score for train set
f1_score(y_DT_test, y_DT_pred)

CPU times: user 3.25 ms, sys: 0 ns, total: 3.25 ms
Wall time: 11.1 ms


0.6666666666666666

In [22]:
%%time
print("Classification report:")
print(classification_report(y_DT_test, y_DT_pred))

Classification report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        71
           1       0.67      0.67      0.67         9

    accuracy                           0.93        80
   macro avg       0.81      0.81      0.81        80
weighted avg       0.93      0.93      0.93        80

CPU times: user 10.7 ms, sys: 888 µs, total: 11.5 ms
Wall time: 23 ms


In [23]:
%%time
df_test=pd.DataFrame(data=test)
df_test=df_test[new_columns]

CPU times: user 70.7 ms, sys: 60 µs, total: 70.7 ms
Wall time: 85 ms


In [24]:
%%time
#predicted the test set values
test_DT_pred=list(DT.predict(df_test))

CPU times: user 208 ms, sys: 2.88 ms, total: 211 ms
Wall time: 224 ms


In [24]:
%%time
file=open('/content/drive/MyDrive/Colab Notebooks/result_DT_var1.txt','w')
for i in test_DT_pred:
  file.write(str(i)+'\n')
file.close()
#f1-score for test set came out to be 0.6

F1-score

Naive bayes -> 0.75
Decison Tree -> 0.60

Naive bayes classifier has given me better f1-score of 0.75